In [ ]:
import torch
import numpy as np
import torch.nn as nn
import os.path as osp

from abl.reasoning import ReasonerBase, KBBase
from abl.learning import BasicNN, ABLModel
from abl.bridge import SimpleBridge
from abl.evaluation import SymbolMetric, SemanticsMetric
from abl.utils import ABLLogger, print_log

from examples.models.nn import SymbolNet
from datasets.get_hwf import get_hwf

In [ ]:
# Initialize logger and print basic information
print_log("Abductive Learning on the HWF example.", logger="current")

# Retrieve the directory of the Log file and define the directory for saving the model weights.
log_dir = ABLLogger.get_current_instance().log_dir
weights_dir = osp.join(log_dir, "weights")

### Logic Part

In [ ]:
# Initialize knowledge base and reasoner
class HWF_KB(KBBase):
    def _valid_candidate(self, formula):
        if len(formula) % 2 == 0:
            return False
        for i in range(len(formula)):
            if i % 2 == 0 and formula[i] not in ["1", "2", "3", "4", "5", "6", "7", "8", "9"]:
                return False
            if i % 2 != 0 and formula[i] not in ["+", "-", "times", "div"]:
                return False
        return True

    def logic_forward(self, formula):
        if not self._valid_candidate(formula):
            return np.inf
        mapping = {str(i): str(i) for i in range(1, 10)}
        mapping.update({"+": "+", "-": "-", "times": "*", "div": "/"})
        formula = [mapping[f] for f in formula]
        return eval("".join(formula))


kb = HWF_KB(
    pseudo_label_list=["1", "2", "3", "4", "5", "6", "7", "8", "9", "+", "-", "times", "div"],
    max_err=1e-10,
    use_cache=False,
)
reasoner = ReasonerBase(kb, dist_func="confidence")

### Machine Learning Part

In [ ]:
# Initialize necessary component for machine learning part
cls = SymbolNet(num_classes=len(kb.pseudo_label_list), image_size=(45, 45, 1))
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(cls.parameters(), lr=0.001, betas=(0.9, 0.99))

In [ ]:
# Initialize BasicNN
# The function of BasicNN is to wrap NN models into the form of an sklearn estimator
base_model = BasicNN(
    model=cls,
    criterion=criterion,
    optimizer=optimizer,
    device=device,
    save_interval=1,
    save_dir=weights_dir,
    batch_size=128,
    num_epochs=3,
)

In [ ]:
# Initialize ABL model
# The main function of the ABL model is to serialize data and
# provide a unified interface for different machine learning models
model = ABLModel(base_model)

### Metric

In [ ]:
# Add metric
metric_list = [SymbolMetric(prefix="hwf"), SemanticsMetric(kb=kb, prefix="hwf")]

### Dataset

In [ ]:
# Get training and testing data
train_data = get_hwf(train=True, get_gt_pseudo_label=True)
test_data = get_hwf(train=False, get_gt_pseudo_label=True)

### Bridge Machine Learning and Logic Reasoning

In [ ]:
bridge = SimpleBridge(model=model, reasoner=reasoner, metric_list=metric_list)

### Train and Test

In [ ]:
bridge.train(train_data, loops=3, segment_size=1000, save_interval=1, save_dir=weights_dir)
bridge.test(test_data)